# Mushroom Classification using Random Forest
Dataset mình sử dụng được lấy từ Kaggle: [Mushroom Classification](https://www.kaggle.com/datasets/uciml/mushroom-classification)    

### Get to know the dataset
Mushroom Classification là dataset gồm 8124 mẫu nấm, được phân thành 2 loại là ăn được (e) và có độc (p). Mỗi mẫu có tổng cộng 22 features và không có giá trị nào bị thiếu (missing value).     
Đầu tiên, chúng ta cần install ```tensorflow_decision_forests``` (TF-DF) package, ```wurlitzer``` package (để hiển thị training log) và import các thư viện cần thiết.

In [ ]:
pip install tensorflow_decision_forests

In [ ]:
pip install wurlitzer

In [21]:
import numpy as np
import matplotlib as plt
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import math

In [ ]:
dataset_df = pd.read_csv("/content/mushrooms.csv")
dataset_df.head(10)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


### Data Pre-processing
TF-DF có thể xử lý các giá trị dạng số và dạng phân loại nên chúng ta không cần one-hot encoding (chuyển các giá trị về 0 và 1). Tuy nhiên, Keras cần giá trị các label là số nguyên nên cần đưa giá trị 2 class ```e``` và ```p``` về 0 và 1.

In [ ]:
label = "class"
classes = dataset_df[label].unique().tolist()
print(f"Classes: {classes}") # Classes: ['p', 'e']
# map the index of 'p' and 'e' to 0 and 1
dataset_df[label] = dataset_df[label].map(classes.index)


Classes: [0, 1]


In [ ]:
# Split the dataset into a training and a testing set
def split_dataset(dataset, test_ratio=0.20):
    test_indices = np.random.rand(len(dataset)) <= test_ratio
    return dataset[~test_indices], dataset[test_indices]

train_ds_df, test_ds_df = split_dataset(dataset_df)
print(f"{len(train_ds_df)} for training, {len(test_ds_df)} for testing")

6509 for training, 1615 for testing


In [22]:
# Convert the pandas dataframe into tensorflow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_df, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_df, label=label)


### Train the model
Đặt ```verbose=2``` để hiển thị training log.

In [23]:
model = tfdf.keras.RandomForestModel(verbose=2)
model.fit(train_ds)

Use 2 thread(s) for training
Use /tmp/tmp7bndf50o as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'cap-shape': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'cap-surface': <tf.Tensor 'data_1:0' shape=(None,) dtype=string>, 'cap-color': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'bruises': <tf.Tensor 'data_3:0' shape=(None,) dtype=string>, 'odor': <tf.Tensor 'data_4:0' shape=(None,) dtype=string>, 'gill-attachment': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'gill-spacing': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>, 'gill-size': <tf.Tensor 'data_7:0' shape=(None,) dtype=string>, 'gill-color': <tf.Tensor 'data_8:0' shape=(None,) dtype=string>, 'stalk-shape': <tf.Tensor 'data_9:0' shape=(None,) dtype=string>, 'stalk-root': <tf.Tensor 'data_10:0' shape=(None,) dtype=string>, 'stalk-surface-above-ring': <tf.Tensor 'data_11:0' shape=(None,) dtype=string>, 'stalk-surface-below-ring': <tf.Tensor 'data_12:0' shape=(None

I0000 00:00:1743412299.562933    1258 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1743412299.563045    1258 kernel.cc:783] Collect training examples
I0000 00:00:1743412299.563080    1258 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1743412299.563526    1258 kernel.cc:401] Number of batches: 7
I0000 00:00:1743412299.563558    1258 kernel.cc:402] Number of examples: 6522
I0000 00:00:1743412299.570100    1258 data_spec_inference.cc:354] 1 item(s) have been pruned (i.e. they are considered out of dictionary) for the column cap-shape (5 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:17434

Model trained in 0:00:00.894531
Compiling model...
Model compiled.


### Model Evaluation

In [ ]:
model.compile(
    metrics=["accuracy"]
)
evaluation = model.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")

2/2 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


In [ ]:
model.make_inspector().evaluation()

Evaluation(num_examples=6454, accuracy=1.0, loss=0.0005675206134897528, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)

Có thể thấy model perform rất tốt trên tập test, với accuracy lên đến 100% và loss chỉ 0.05%.      
Dưới đây là visualization quá trình train 1 trong số 300 cây trong ensemble.

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=2, max_depth=5)

### Using YDF instead of TF-DF
YDF là phiên bản nâng cấp của TF-DF, đơn giản hóa implementation và cải thiện performance của model. Ngoài ra, YDF có thể làm việc trực tiếp với pandas dataframe mà không cần chuyển sang tensorflow dataset.

In [25]:
import ydf

model_ = ydf.RandomForestLearner(label=label, num_trees=300).train(train_ds_df)

Train model on 6522 examples
Model trained in 0:00:01.117159


In [26]:
# Evaluate the model
model_.evaluate(test_ds_df)

Label \ Pred,0,1
0,760,0
1,0,842
